In [8]:
import zmq
import numpy as np
import time
import matplotlib.pyplot as plt

from rfsoc_qsfp_offload.overlay import Overlay
ol = Overlay(ignore_version=True)



context = zmq.Context()
socket = context.socket(zmq.SUB)
socket.connect("tcp://10.29.234.35:50241")
# socket.connect("tcp://hay-rfsoc-003.mit.edu:55555") # connect, not bind, the PUB will bind, only 1 can bind
socket.setsockopt(zmq.SUBSCRIBE, b'') # subscribe to topic of all (needed or else it won't work)
dtype = np.dtype([('r', '<i2'), ('i', '<i2')])



In [9]:
DAC_TILE = 0       # DAC Tile 228
DAC_BLOCK = 0       # DAC Block 0
DAC_SAMPLE_FREQUENCY = 4915.2/8#4915.2  # MSps
DAC_PLL_FREQUENCY = 491.52   # MHz
DAC_FC = 0.0

ol.initialise_dac(tile=DAC_TILE,
                  block=DAC_BLOCK,
                  pll_freq=DAC_PLL_FREQUENCY,
                  fs=DAC_SAMPLE_FREQUENCY,
                  fc=DAC_FC
                 )

In [10]:

from pynq import allocate



first_buf = True

time_count = 0
while True:
    if socket.poll(10) != 0: # check if there is a message on the socket
        time_count=0
        msg = socket.recv() # grab the message
        txdata = np.frombuffer(msg, dtype=np.dtype('<i2'), count=-1)
        print("Tx out") # size of msg
        if first_buf:
            tx_buffer = allocate(shape=(txdata.size,), dtype=np.dtype('<i2'))
            first_buf=False
        ol.axi_dma_dac.sendchannel.transfer(tx_buffer)
        time.sleep(10.)
        ol.axi_dma_dac.sendchannel.stop()
        
    else:
        time.sleep(0.1)
        time_count+=1
        if time_count >1000:
            if not first_buf:
                ol.axi_dma_dac.sendchannel.stop()
            break
    
            

Tx out
Tx out
Tx out
Tx out
Tx out
Tx out
Tx out
Tx out
Tx out
Tx out


KeyboardInterrupt: 

ol.axi_dma_dac.sendchannel.stop()

In [11]:
ol.axi_dma_dac.sendchannel.stop()

In [12]:
ol.axi_dma_dac.sendchannel.stop()